#API 사용해보기

In [ ]:
# OPEN API 활용가이드 (법제처)
# https://open.law.go.kr/LSO/openApi/guideList.do

## 현행법령 본문조회 설명(법제처) (공표기준)
## https://open.law.go.kr/LSO/openApi/guideResult.do


## 시행법령 본문조회 설명(법제처) (시행기준) - 법령정보센터와 동일 내용 조회 가능
## https://www.law.go.kr/LSO/openApi/guideResult.do

##  XML - 링크 정보 없음


In [1]:
import requests
from urllib.request import urlopen
from urllib.parse import urlencode,unquote,quote_plus
import urllib

from bs4 import BeautifulSoup

In [41]:
#!pip install lxml
!pip show lxml

Name: lxml
Version: 4.6.1
Summary: Powerful and Pythonic XML processing library combining libxml2/libxslt with the ElementTree API.
Home-page: https://lxml.de/
Author: lxml dev team
Author-email: lxml-dev@lxml.de
License: BSD
Location: c:\users\user\anaconda3\lib\site-packages
Requires: 
Required-by: 


In [187]:
# 법령 XML 상세조회 (시행기준. target=eflaw)

#기본 url
url_base = " http://www.law.go.kr/DRF/lawService.do"

#조회 url 세부 설정
user_id = 'bmsong' #open API ID (bmsong@kau.ac.kr의 아이디 부분)
target = "eflaw" #법령 조회시
MST = "218289" #ID 또는 MST #산업안전보건법 MST 218289
#ID = "001766" 
            #ID 산업안전보건법 법령ID 001766 (설명에는 ID로 조회하면 현행법령이 조회된다는데, 
            #target을 eflaw로 하면 ID로 요청해도, MST로 요청해도 같은 것 같다.
Type = "XML" #출력 형태 : HTML 또는 XML

url_sub = "?" +\
        "OC=" + user_id +\
        "&target=" + target +\
        "&MST=" + MST +\
        "&type=" + Type

#최종 url
url = url_base + url_sub

print(url)

response = requests.get(url)

#(1) response.text 사용하는 방법
#soup = BeautifulSoup(response.text.encode('utf-8'), 'lxml-xml') #lxml-xml -> lxml 추가 설치 필요(Beautifulsoup과 별도로 설치)

#(2) response.content 사용하는 방법
soup = BeautifulSoup(response.content, 'lxml-xml') #lxml-xml -> lxml 추가 설치 필요(Beautifulsoup과 별도로 설치)


#soup

 http://www.law.go.kr/DRF/lawService.do?OC=bmsong&target=eflaw&MST=218289&type=XML


In [45]:
# requests 객체로 페이지 내용 가져올 때, .text와 .content의 차이
# https://blog.ch4n3.kr/515
# 1).text는 읽을 수 있는 텍스트를 가져오고, encoding을 추측 또는 지정할 수 있음. (.text.encode('utf-8') 등의 형태로)
# 2) .content는 바이트 형태로 그대로 가져오므로, 이미지 등 텍스트 아닌 내용도 가져올 수 있음. 
# r.content를 사용해야 하는 이유?
# r.encoding = 'utf-8' 을 사용하여 r.text를 가져오는 경우는 너무나 비범용적이다. 
# 따라서 이 경우 r.content를 사용하면 한글 데이터를 보존할 수 있게 된다.

In [62]:
#soup

In [189]:
#산업안전보건법 - 기본 정보 dataframe 저장 
#시행일자
import pandas as pd
import numpy as np
lc_basic_df = pd.DataFrame() #DataFrame for Law Content 

In [200]:
data = soup.find('법령ID') #MST 정보
lc_basic_df['법령ID'] = [data.get_text()]

data = soup.find('법종구분') 
lc_basic_df['법종구분']= [data.get_text()]

data = soup.find('법령명_한글') 
lc_basic_df['법령명']= [data.get_text()]

data = soup.find('시행일자')
lc_basic_df['시행일자'] = [data.get_text()]

data = soup.find('공포일자') 
lc_basic_df['공포일자']= [data.get_text()]

data = soup.find('소관부처') 
lc_basic_df['소관부처']= [data.get_text()]

In [201]:
lc_basic_df

,법령ID,법종구분,법령명,시행일자,공포일자,소관부처
0,001766,법률,산업안전보건법,20210116,20200526,고용노동부


In [202]:
#법령기본정보 내보내기
#데이터 내보내기
#csv로 내보내기
lc_basic_df.to_csv('../data/data_210522/00. lc_basic_df(bomi)_산업안전보건법.csv', sep=',', encoding='utf-8')

#피클로 내보내기
lc_basic_df.to_pickle("../data/data_210522/00. lc_basic_df(bomi)_산업안전보건법.pkl")

# 참고 문법: load (피클 불러오기)
#d = pd.read_pickle("../data/00. lc_df(bomi)_산업안전보건법_조.pkl")

In [ ]:
#조문내용 DataFrame으로 저장
import pandas as pd
import numpy as np
lc_df = pd.DataFrame() #DataFrame for Law Content 


In [170]:
#조문키 가져오기
data = soup.find_all('조문단위')
lc_df['조문키']=[d['조문키'] for d in data] #<조문단위>의 '조문키=' 속성 값 가져오기

#조문번호 가져오기
data = soup.find_all('조문번호')
lc_df['조문번호']=[d.get_text() for d in data]
#조문번호 2개씩 있는 것들 이유 => 각 장의 첫 번째 조, 전문에 대한 내용이 있기 때문
#조문번호, 조문여부(조문 or 전문)을 가져와서 DataFrame으로 만든 후, 전문은 제외하는 방식 가능

#조문여부 가져오기
data = soup.find_all('조문여부')
lc_df['조문여부']=[d.get_text() for d in data]

#조문제목 가져오기
data = soup.find_all('조문제목')
lc_df['조문제목']=[d.get_text() for d in data]

#조문내용 가져오기
#<조문단위>태그 안에 <조문내용><호내용><항내용><목내용> 등 <내용>으로 끝나는 태그 안 내용을 합치기
#정규식 활용
import re
data = soup.find_all('조문단위')

tmps = [d.find_all(re.compile ("내용$")) for d in data] #<내용>으로 끝나는 태그 가져오기 (조문내용, 호내용, 항내용, 목내용)

results=list()
for tmp in tmps : 
    ts =[t.get_text() for t in tmp]
    re = "\n".join(ts)
    results.append(re)

lc_df['조문내용'] = results


In [179]:
#조문번호는 같고, <조문가지번호>에서 구별되는 것들 확인 cell
lc_df[lc_df['조문여부']=='조문']['조문번호'] 

# 조 총 177개, 175조 + 166조의 2 + 170조의 2
# ~조의 2는 <조문단위>안에 <조문가지번호> 태그가 있으면, ~의 2

1        1
2        2
3        3
4        4
5        5
      ... 
194    171
195    172
196    173
197    174
198    175
Name: 조문번호, Length: 177, dtype: object

In [183]:
#조문번호는 같고, <조문가지번호>에서 구별되는 것들을 위한 '조문번호/가지번호' 컬럼추가
#<조문가지>가 없으면, 조문번호와 같고, 조문가지번호가 있으면, 조문번호/조문가지번호 (166/2는 제166조의2를 의미)
#조문제목 가져오기
data = soup.find_all('조문단위')
tmp_list=[]
for d in data :
    if d.find('조문가지번호') : 
        tmp = d.find('조문번호').get_text() + "/" + d.find('조문가지번호').get_text()
    else : 
        tmp = d.find('조문번호').get_text()
    tmp_list.append(tmp)

lc_df['조문번호/가지번호']=tmp_list

In [184]:
lc_df

,조문키,조문번호,조문여부,조문제목,조문내용,조문번호/가지번호
0,0001000,1,전문,,\n\t\t\t제1장 총칙\n,1
1,0001001,1,조문,목적,제1조(목적) 이 법은 산업 안전 및 보건에 관한 기준을 확립하고 그 책임의 소재를...,1
2,0002001,2,조문,정의,제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020.5.2...,2
3,0003001,3,조문,적용 범위,"제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 정도, 사업...",3
4,0004001,4,조문,정부의 책무,제4조(정부의 책무)\n ① 정부는 이 법의 목적을 달성하기 위하여 다음 각 호의 ...,4
...,...,...,...,...,...,...
194,0171001,171,조문,벌칙,제171조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 1천만원 이하의 벌금에 ...,171
195,0172001,172,조문,벌칙,제172조(벌칙) 제64조제1항 또는 제2항을 위반한 자는 500만원 이하의 벌금에...,172
196,0173001,173,조문,양벌규정,"제173조(양벌규정) 법인의 대표자나 법인 또는 개인의 대리인, 사용인, 그 밖의 ...",173
197,0174001,174,조문,형벌과 수강명령 등의 병과,제174조(형벌과 수강명령 등의 병과)\n ① 법원은 제38조제1항부터 제3항까지(...,174


In [186]:
#데이터 내보내기
#csv로 내보내기
lc_df.to_csv('../data/data_210522/00. lc_df(bomi)_산업안전보건법_조.csv', sep=',', encoding='utf-8')

#피클로 내보내기
lc_df.to_pickle("../data/data_210522/00. lc_df(bomi)_산업안전보건법_조.pkl")

# 참고 문법: load (피클 불러오기)
#d = pd.read_pickle("../data/00. lc_df(bomi)_산업안전보건법_조.pkl")

In [108]:
#테스트용 cell
data = soup.find_all('조문단위')
data[1].find('조문내용').get_text()

'제1조(목적) 이 법은 산업 안전 및 보건에 관한 기준을 확립하고 그 책임의 소재를 명확하게 하여 산업재해를 예방하고 쾌적한 작업환경을 조성함으로써 노무를 제공하는 사람의 안전 및 보건을 유지ㆍ증진함을 목적으로 한다. <개정 2020.5.26>'

In [171]:
#확인용 cell
lc_df

,조문키,조문번호,조문여부,조문제목,조문내용
0,0001000,1,전문,,\n\t\t\t제1장 총칙\n
1,0001001,1,조문,목적,제1조(목적) 이 법은 산업 안전 및 보건에 관한 기준을 확립하고 그 책임의 소재를...
2,0002001,2,조문,정의,제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020.5.2...
3,0003001,3,조문,적용 범위,"제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 정도, 사업..."
4,0004001,4,조문,정부의 책무,제4조(정부의 책무)\n ① 정부는 이 법의 목적을 달성하기 위하여 다음 각 호의 ...
...,...,...,...,...,...
194,0171001,171,조문,벌칙,제171조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 1천만원 이하의 벌금에 ...
195,0172001,172,조문,벌칙,제172조(벌칙) 제64조제1항 또는 제2항을 위반한 자는 500만원 이하의 벌금에...
196,0173001,173,조문,양벌규정,"제173조(양벌규정) 법인의 대표자나 법인 또는 개인의 대리인, 사용인, 그 밖의 ..."
197,0174001,174,조문,형벌과 수강명령 등의 병과,제174조(형벌과 수강명령 등의 병과)\n ① 법원은 제38조제1항부터 제3항까지(...
